In [ ]:
from isimip_client.client import ISIMIPClient
client = ISIMIPClient()

In [ ]:
# to cut out a bounding box in lat/lon use
# run this subsequently to poll the status
path = 'ISIMIP3a/SecondaryInputData/climate/atmosphere/obsclim/global/daily/historical/CHELSA-W5E5v1.0/chelsa-w5e5v1.0_obsclim_tas_30arcsec_global_daily_201601.nc'
response = client.cutout(path, bbox=[-45.108, -41.935, 167.596, 173.644])
response

In [ ]:
# once the status is 'finished', get the url to download the result
client.download(response['file_url'], path='downloads', validate=False, extract=True)

In [ ]:
# plot the first timestep of the file
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from pathlib import Path

path = Path.cwd() / 'downloads' / 'chelsa-w5e5v1.0_obsclim_tas_30arcsec_lat-45.108to-41.935lon167.596to173.644_daily_201601.nc'
with Dataset(path) as ds:
    x = ds.variables['lon'][:]
    y = ds.variables['lat'][:]
    z = np.flip(ds['tas'][0, :, :], 0)
    
    img = np.flip(ds['tas'][0, :, :], 0)
    plt.imshow(z, extent=[x[0], x[-1], y[0], y[-1]], cmap='coolwarm', aspect='auto')
    plt.colorbar()
    plt.gcf().set_size_inches(12, 12)

In [ ]:
# To mask a specific dataset, the following code can be used to first search the
# repository and then mask every file in the dataset. After starting the masking job 
# (on the server) it checks every 10 seconds, if the job is done and then downloads
# the file. Masking the files on the server can take a few minutes or even longer.

# get the dataset metadata from the ISIMIP repository
response = client.datasets(simulation_round='ISIMIP3a',
                           product='SecondaryInputData',
                           climate_forcing='chelsa-w5e5v1.0',
                           climate_scenario='obsclim',
                           climate_variable='tas')

dataset = response["results"][0]
paths = [file['path'] for file in dataset['files']]

# start/poll a masking job on the server (e.g. for China)
response = client.cutout(paths, [-45.108, -41.935, 167.596, 173.644], poll=10)

# download the file when it is ready
client.download(response['file_url'], path='downloads', validate=False, extract=True)